# Check against DIVD DB

Check if the NLM Hashes from the LEAK specific database are already in the DIVD database

# Set constants

In [ ]:
CASE="DIVD-2021-00012"
SUB="set1"

# Keep these the same
IN_DIR="../LEAK_DB/"
IN_DB=f"{IN_DIR}/{CASE}-{SUB}.sqlite3"
DIVD_DB="../SHARED_DBS/DIVD-leaks.sqlite3"

# Import helper functions

In [ ]:
%run ../0.shared_notebooks/0_helper_functions.ipynb

# Open DB

In [ ]:
divd_conn=sqlite3.connect(DIVD_DB)


In [ ]:
divd_conn


In [ ]:
in_conn=sqlite3.connect(IN_DB)

In [ ]:
in_conn

# Check hashes

In [ ]:
in_cur = in_conn.cursor()
in_cur.execute("SELECT COUNT( DISTINCT nml_hash) from entity;" )
row = in_cur.fetchone()
todo=row[0]
count = 0
# Progress bar
widgets=[
    progressbar.SimpleProgress(), 
    ' (', progressbar.Percentage(format='%(percentage)3.2f%%'), ') ',
    progressbar.Bar(),
    progressbar.ETA(),
]
bar = progressbar.ProgressBar(max_value=todo, widgets=widgets, enable_colors=False)
bar.update(1)
for row in in_cur.execute("SELECT DISTINCT nml_hash FROM entity").fetchall() :
    if row[0] :
        for lrow in divd_conn.execute("""
            SELECT l.case_id, l.sub_id, l.description, l.date 
            FROM divd d 
            LEFT JOIN leak l 
                ON d.case_id = l.case_id AND d.sub_id = l.sub_id
            WHERE d.nml_hash = ? AND d.case_id <> ? AND d.sub_id <> ?;
        """,
            ( row[0], CASE, SUB )
        ).fetchall() :
            in_cur.execute(
                "INSERT INTO divd ( nml_hash, case_id, sub_id, description, date ) values ( ?, ?, ?, ?, ? );",
                ( row[0], lrow[0], lrow[1], lrow[2], lrow[3] )
            )
    count=count+1
    if count % 100 == 0 :
        bar.update(count)
    if count % 10_000 == 0 :
        divd_conn.commit()
divd_conn.commit()
bar.update(todo)
print("Done")


In [ ]:
count